# Librairies

In [15]:
import os
import numpy as np

# Intake
from dotenv import load_dotenv
from intake import open_catalog

# Pandas
import pandas as pd
import geopandas  as gpd

# H3
import h3
import h3pandas
from tobler.util import h3fy
from tobler.area_weighted import area_interpolate, area_join

# SQL
# import libpysal
from sqlalchemy import *
from geoalchemy2 import Geometry, WKTElement

# Dask
# from dask import dataframe as dd_df
# import dask_geopandas
import dask_geopandas as ddg
from dask.distributed import Client

# Environnement

In [2]:
load_dotenv()

usr=os.getenv("DB_USER")
pswd=os.getenv("DB_PWD")
host=os.getenv("DB_HOST")
port=os.getenv("DB_PORT")
home=os.getenv("HOME_PATH")
db_traitement=os.getenv("DB_WORKSPACE")
db_ref=os.getenv("DB_REF")
db_externe=os.getenv("DB_EXT")
dwh_fact_strategy=os.getenv("DWH_FACT_STRATEGY")
dwh_dim_strategy=os.getenv("DWH_DIM_STRATEGY")


commun_path = os.getenv("COMMUN_PATH")
project_dir = os.getenv("PROJECT_PATH")
data_catalog_dir = os.getenv("DATA_CATALOG_DIR")
data_output_dir = os.getenv("DATA_OUTPUT_DIR")
sig_data_path = os.getenv("SIG_DATA_PATH")
db_workspace = os.getenv("DB_WORKSPACE")
db_workspace = os.getenv("DB_REF")

# Fonctions

## Fonctions de traitements

In [21]:
"""
Fonction retournant un DataFrame après ajout d'une colonne hex_id en index.
En d'autres termes, cette fonction indexe un GeoDataFrame sur une grille uniforme.

param gdf: GeoDataFrame en entrée
param resolution: résolution des hexagones
return: DataFrame

"""
def indexation(gdf, resolution):
    
    # Mise en conformité de la colonne "geometry"
    if(gdf.geometry.name != 'geometry'):
                gdf.rename_geometry('geometry',inplace=True)

    gdf = gdf.to_crs(epsg=4326).h3.polyfill(resolution, explode=False) # Indexation du DataFrame
    df = pd.DataFrame(gdf.drop(columns={gdf.geometry.name})) # Suppression de la colonne "geometry"
    df = df.explode('h3_polyfill') # Explosion des liste d'identifiants hexagonaux
    df.rename(columns={'h3_polyfill':'hex_id'}, inplace=True) # Renommage de la colonne hex_id
    df.set_index('hex_id', inplace=True) # Définition de l'index
    
    return df

In [75]:
"""
Fonction permettant l'indexation d'un GeodataFrame sur une grille adaptative.

param gdf: GeoDataFrame en entrée
param list_col: liste des champs classifiants
param tx_theme: dictionnaire {résolution:taux} des taux à partir du quel un élément classifiant est considéré comme unique 
    dans l'hexagone considéré (entre 0 et 1) pour une résolution donnée
param tx_spatial: taux minimal de remplissage d'une zone de données
param res_min: résolution minimale des hexagones
param res_max: résolution maximale des hexagones
param output: paramètre servant uniquement à la récursivité de la fonction (à laisser égal à None)
param geom_clip: paramètre servant uniquement à la récursivité de la fonction (à laisser égal à None)
return: DataFrame indexé

"""
def compact(gdf, list_col, tx_theme, tx_spatial, res_min=5, res_max=12, output=None, geom_clip=None):
    
    # Création d'un GeoDataFrame vide
    if(output is None):
        output = gpd.GeoDataFrame(columns=list_col+['geometry'], geometry='geometry')
        output.index.names = ['hex_id']
        
    # Récupération des géométries des hexagones à la résolution donnée
    if(geom_clip is None):
        clip = h3fy(gdf, resolution=res_min).reset_index()
    else:
        clip = h3fy(geom_clip, resolution=res_min).reset_index() 
    
    # Condition d'arrêt de la récursivité
    if(res_min <= res_max):
        print('Résolution ' +  str(res_min))
        
        valid_cells = [] # Cellules n'ayant pas besoin d'être divisées
        valid_label = [] # Classe associée à la cellule
        labels = gdf[list_col[0]].unique() # Liste des différentes classes possibles

        # Association des taux d'occupation de chaque classe pour chaque hexagone
        gdf_interp = area_interpolate(source_df=gdf, target_df=clip, categorical_variables=list_col)
        # Peuplement de la colonne hex_id par jointure avec la table des géométries
        gdf_interp['hex_id'] = gdf_interp.join(clip, rsuffix='clip')['hex_id']
        gdf_interp.set_index('hex_id', inplace=True)
        
        # Pour chaque enregistrement
        for index, row in gdf_interp.iterrows():
            for i in range(gdf_interp.columns.size-1): # Pour chaque colonne (sauf la géométrie)
                if(res_min < res_max): # Si la résolution maximale n'est pas atteinte
                    # Si le pourcentage de présence de la classe dans la cellule est supérieure à tx_spatial et est supérieure à tx_theme de l'ensemble des classes
                    if((row[gdf_interp.columns][i] >= tx_theme[res_min] * row[gdf_interp.columns][:-1].sum()) & (row[gdf_interp.columns][i] >= tx_spatial)):
                        valid_cells.append(index)
                        nom_colonne = gdf_interp.columns[i]
                        valid_label.append(labels[gdf_interp.columns.get_loc(nom_colonne)])
                        break
                else: # Si la résolution maximale est atteinte
                    # La valeur de cellule est déterminée par la classe majoritaire même si tx_theme n'est pas atteint
                    if(row[gdf_interp.columns][i] == max(row[gdf_interp.columns][:-1])):
                        valid_cells.append(index)
                        nom_colonne = gdf_interp.columns[i]
                        valid_label.append(labels[gdf_interp.columns.get_loc(nom_colonne)])
                        break

        
        if(valid_cells): # Si des cellules sont compactes
            # Création d'un GeoDataFrame contenant les cellules valides avec leur classe associée
            gdf_valid = gdf_interp.loc[valid_cells]
            data = {list_col[0]: valid_label, 'geometry': gdf_valid['geometry']}
            gdf_valid_bdd = gpd.GeoDataFrame(data, geometry='geometry', crs="EPSG:3163", index=gdf_valid.index)
            # Concaténation des cellules compactées de résolution plus faible avec les nouvelles cellules
            output = pd.concat([output, gdf_valid_bdd], ignore_index=False) 
        
        if(res_min < res_max): # Si la résolution max n'est pas atteinte
            if(valid_cells): # Génération d'une géométrie excluant les cellules valides
                if(geom_clip is None):
                    geom_clip = gpd.GeoDataFrame({'geometry':gdf.geometry.unary_union.difference(gdf_valid.geometry.unary_union)}, geometry='geometry', crs="EPSG:3163")
                else:
                    geom_clip = gpd.GeoDataFrame({'geometry':geom_clip.geometry.unary_union.difference(gdf_valid.geometry.unary_union)}, geometry='geometry', crs="EPSG:3163")
                return agregInv(gdf, list_col, tx_theme, tx_spatial, res_min+1, res_max, output, geom_clip) # Récursion suivante à une résolution supérieure
            else:
                if(geom_clip is None):
                    return agregInv(gdf, list_col, tx_theme, tx_spatial, res_min+1, res_max, output) # Récursion suivante à une résolution supérieure
                else:
                    return agregInv(gdf, list_col, tx_theme, tx_spatial, res_min+1, res_max, output, geom_clip) # Récursion suivante à une résolution supérieure
        else:
            output = pd.DataFrame(output.drop(columns={output.geometry.name})) # Suppression de la colonne des géométries
            return output

## Fonctions BDD

In [22]:
"""
Fonction retournant l'engine de connexion à la base de données.

param user: user
param pswd: mot de passe
param host: hôte
param dbase: nom de la base de données

"""
def getEngine(user=usr, pswd=pswd, host=host, dbase=db_traitement):
    connection = f'postgresql://{user}:{pswd}@{host}:{port}/{dbase}'
    return create_engine(connection)

In [23]:
"""
Fonction d'intégration ou de mise à jour des données dans le DWH.

param new_lines: DataFrame contenant les données à intégrer
param table_name: nom de la table de destination 
    (en mode 'append' si la table est inexistante ou en mode 'replace', une nouvelle table sera créee)
param engine: engine de connexion à la base de données
param schema: schéma dans lequel se trouve la table de destination
param methode: 'append' pour ajouter des données à une table déjà existante ou 
    'replace' pour écraser la table de destination si elle existe déjà
param geom: Boolean indiquant si les données contiennent une dimension géométrique
param dtype: dictionnaire {nom_champ:type} permettant d'indique le type de certains champs

"""

def updateTable(new_lines, table_name, engine, schema, methode='append', geom=True, dtype=None):
    dict_types = {'geometry': Geometry('POLYGON', srid=3163)}
    
    if(methode=='replace'): # Suppression de la table de destination si elle existe déjà
        engine.execute(f'DROP TABLE IF EXISTS {schema}.{table_name} CASCADE')
    if(dtype is not None): # Mise à jour du dictionnaire des types de champs
        dict_types.update(dtype)
        
    if(not new_lines.empty): # Si des données sont à integrer
        if(geom): # Si les données contiennent une dimension géométrique
            if(new_lines.geometry.name != 'geom'):
                new_lines = new_lines.rename_geometry('geom')
            new_lines['geometry'] = new_lines['geom'].apply(lambda x: WKTElement(x.wkt, srid=3163))
            new_lines.drop('geom', 1, inplace=True)
            new_lines.to_sql(name=table_name, con=engine, schema=schema, if_exists=methode, index=True, index_label='hex_id', dtype=dict_types)
        else:
            new_lines.to_sql(name=table_name, con=engine, schema=schema, if_exists=methode, index=True, index_label='hex_id', dtype=dict_types)
    return new_lines

In [41]:
"""
Génération d'une vue ajoutant une colonne de géométrie à la table donnée en argument.

param table_name: nom de la table
param engine: engine de connexion à la base de données
param schema: schéma

"""
def geomView(table_name, engine, schema):
    query = f'DROP VIEW IF EXISTS {schema}.view_{table_name};' + f'CREATE VIEW {schema}.view_{table_name} AS (SELECT *, h3_to_geo_boundary(hex_id::h3index)::geometry AS geometry FROM {schema}.{table_name})'
    engine.execute(query)

## Fonctions Dask

In [32]:
client = Client('192.168.1.44:8786')

/opt/anaconda3/envs/gis_39/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.21.4 | 1.21.4    | 1.20.3  |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [33]:
client

<Client: 'tcp://172.17.0.3:8786' processes=1 threads=12, memory=31.85 GiB>

In [ ]:
def indexation_dask(gdf, npartitions, *iterables):
    
    df_meta = gpd.GeoDataFrame(columns=gdf.columns)
    
    data = ddg.from_geopandas(gdf,npartitions)
    gdf_map = data.map_partitions(func=indexation, args=iterables, meta=df_meta)
    client.persist(gdf_map)
    gdf_map.compute()

In [ ]:
indexation_dask(gdf=data_communes, npartitions=2, resolution=8)

## Fonctions utiles

In [25]:
"""
Fonction permettant de charger une table sous forme de DataFrame à partir 
d'un catalogue Intake.

param catalog: catalogue intake
param table_name: nom de la table référencé dans le catalogue
return: DataFrame

"""

def loadData(catalog, table_name):
    dataName = f"{table_name}"
    entryCatalog = getattr(open_catalog(catalog),dataName)
    data = entryCatalog
    return data.read()

In [26]:
"""
Fonction permettant de remplacer les valeurs d'un ou plusieurs champ(s) 
d'une table par les valeurs d'un champ d'une autre table (appelé champ 
de standardisation) suivant une jointure définie.

param df: DataFrame en entrée
df_right: DataFrame de jointure indexé sur son champ de jointure
std_field_right: champ de standardisation de field_right
dic: {join_field_df: champ de jointure de df, num_col: numéro de colonne du futur champ standardisé}
return: DataFrame standardisé

"""

def standardizeField(df, df_right, std_field_right, dic):
    for join_field_df, num_col in dic.items():
        std_field_df = df.join(df_right, on=join_field_df)[std_field_right] # Création du champ standardisé
        df = df.drop(join_field_df, axis= 1) # Suppression du champ non standardisé
        df.insert(num_col, join_field_df, std_field_df, allow_duplicates=True) # Insertion du champ standardisé
    return df

In [91]:
"""
Fonction retourant un dictionnaire associant à chaque résolution le taux à partir duquel un 
élément classifiant est considéré comme unique dans chaque cellule.

param min_carto_unit_m: unité minimale de cartographie (plus petit détail visible) en m
return: dictionnaire

"""
def compute_dict_tx(min_carto_unit_m):
    hex_area=[4250546.8477000, 607220.9782429, 86745.8540347, 12392.2648621, 1770.3235517, 252.9033645, 36.1290521,
              5.1612932, 0.7373276, 0.1053325, 0.0150475, 0.0021496, 0.0003071, 0.0000439, 0.0000063, 0.0000009]
    dic = {}
    for i in range (16):
        val = 1-(((min_carto_unit_m**2)/1000000)/hex_area[i])
        if val >= 0:
            dic[i] = val
        else:
            dic[i] = 0
    return dic

# Données

Récupération des données sources

In [16]:
# Connexion à la base de données "oeil_traitement"
engine = getEngine()

In [17]:
# Connexion à la base de données du RDS
engineRDS = getEngine(user='postgres',pswd='XwUxFfrL6yRK5Wz',host='oeil-pg-aws.cluster-ck8dgtf46vxd.ap-southeast-2.rds.amazonaws.com',dbase='oeil')

In [18]:
catalog = f"{data_catalog_dir}bilbo_data.yaml" # Choix du catalogue de données

In [19]:
# Coordonnées d'une bbox sur l'île des Pins
xmin = 536159
xmax = 571819
ymin = 156515
ymax = 190058

## Tables de dimensions

### communes

In [27]:
%%time
# Récupétation de la table des communes sur l'emprise souhaitée
data_communes = loadData(catalog,'communes') # .cx[xmin:xmax, ymin:ymax]

CPU times: user 3.22 s, sys: 44.8 ms, total: 3.27 s
Wall time: 4.42 s


In [28]:
data_communes # Visualisation de la table

,objectid,nom,nom_minus,code_com,code_post,nom_fichier,shape_length,shape_area,shape
0,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08,MULTIPOLYGON Z (((450513.398 219933.702 -10000...
1,2,LA FOA,La Foa,98813,98880,LA_FOA,298739.047200,4.609553e+08,MULTIPOLYGON Z (((371445.862 277499.216 -10000...
2,3,POUM,Poum,98826,98826,POUM,676002.442656,4.702959e+08,"MULTIPOLYGON Z (((191490.890 456172.812 1.000,..."
3,4,OUEGOA,Ouégoa,98819,98821,OUEGOA,365782.158975,6.491957e+08,MULTIPOLYGON Z (((235001.134 434615.994 -10000...
4,5,MOINDOU,Moindou,98816,98819,MOINDOU,207876.891360,3.214597e+08,MULTIPOLYGON Z (((371445.862 277499.216 -10000...
5,6,SARRAMEA,Sarraméa,98828,98882,SARRAMEA,58079.795040,1.054019e+08,MULTIPOLYGON Z (((376716.288 288761.483 9999.0...
6,7,POUEBO,Pouébo,98824,98824,POUEBO,178372.386245,1.957887e+08,MULTIPOLYGON Z (((272317.912 407594.073 -10000...
7,8,YATE,Yaté,98832,98834,YATE,351071.233169,1.332883e+09,MULTIPOLYGON Z (((456960.568 261512.590 -10000...
8,9,PAITA,Païta,98821,98889,PAITA,398184.267799,6.920473e+08,MULTIPOLYGON Z (((439992.642 225404.891 -10000...
9,10,VOH,Voh,98831,98833,VOH,387363.290562,7.971851e+08,MULTIPOLYGON Z (((266338.429 348818.522 -10000...


### dim_dates

In [129]:
%%time
# Récupétation de la table de standardisation des dates
# Le set_index sur le champ de jointure est nécessaire à l'application de la fonction "standardizeField"
data_date = pd.read_sql("SELECT * FROM pression_eau.dim_date",engine).set_index('date')

CPU times: user 80.2 ms, sys: 380 µs, total: 80.6 ms
Wall time: 157 ms


In [130]:
data_date # Visualisation de la table

,index,date_id,week_day,day,month,week,quarter,year
date,,,,,,,,
1990-01-01,0,19900101,Monday,1,1,1,1,1990
1990-01-02,1,19900102,Tuesday,2,1,1,1,1990
1990-01-03,2,19900103,Wednesday,3,1,1,1,1990
1990-01-04,3,19900104,Thursday,4,1,1,1,1990
1990-01-05,4,19900105,Friday,5,1,1,1,1990
...,...,...,...,...,...,...,...,...
2021-11-05,11631,20211105,Friday,5,11,44,4,2021
2021-11-06,11632,20211106,Saturday,6,11,44,4,2021
2021-11-07,11633,20211107,Sunday,7,11,44,4,2021


## Tables de faits

### incendies_Sentinel

In [131]:
%%time
# Récupétation de la table des incendies sur l'emprise souhaitée
data_feux_raw = loadData(catalog,'incendies_Sentinel').cx[xmin:xmax, ymin:ymax]

CPU times: user 3.84 s, sys: 130 ms, total: 3.97 s
Wall time: 4.91 s


In [132]:
data_feux_raw # Visualisation de la table

,objectid,province,commune,surface_ha,idfusion,classification,begdate,enddate,fs_x,fs_y,...,pentemediane,penteminimale,pentemaximale,tailleincendie,derniere_detection,debutviirs,finviirs,shape_starea__,shape_stlength__,geom
117,15865,PROVINCE SUD,Ile des Pins,1.109230,S19698,Valide,2020-11-26,2020-11-26,557910,172898,...,NaN,NaN,NaN,0ha - 10ha ...,2020-11-26,NaT,NaT,1.109230e+04,879.694444,"POLYGON ((557883.772 172965.653, 557933.753 17..."
122,11385,Province Sud,ILE DES PINS,14.311355,941,Valide,2017-01-06,2017-01-06,547895,178828,...,NaN,NaN,NaN,None,2017-01-06,NaT,NaT,1.431136e+05,5638.296097,"POLYGON ((547722.914 179070.783, 547723.057 17..."
923,13509,Province Sud,ILE DES PINS,3.517956,1126,Valide,2017-11-17,2017-11-17,548112,174638,...,NaN,NaN,NaN,None,2017-11-17,NaT,NaT,3.517956e+04,1519.558478,"POLYGON ((548153.733 174755.040, 548153.877 17..."
1038,14190,PROVINCE SUD,Ile des Pins,1.019392,S11647,Valide,2020-08-13,2020-08-13,548490,176350,...,NaN,NaN,NaN,0ha - 10ha ...,2020-08-13,NaT,NaT,1.019392e+04,999.701704,"POLYGON ((548411.811 176406.448, 548441.802 17..."
1257,14512,PROVINCE SUD,Ile des Pins,1.359050,SGroup_90j_82,Valide,2020-01-26,2020-01-26,559470,170398,...,NaN,NaN,NaN,0ha - 10ha ...,2020-01-26,NaT,NaT,1.359050e+04,619.783560,"POLYGON ((559521.892 170388.482, 559521.893 17..."
1381,16237,PROVINCE SUD,Ile des Pins,3.258223,S23691,Valide,2020-12-21,2020-12-21,550116,165331,...,NaN,NaN,NaN,0ha - 10ha ...,2020-12-21,NaT,NaT,3.258223e+04,1679.541881,"POLYGON ((550120.275 165461.609, 550140.269 16..."
1520,10967,Province Sud,ILE DES PINS,14.191474,942,Valide,2017-01-06,2017-01-16,548398,177246,...,NaN,NaN,NaN,None,2017-01-16,NaT,NaT,1.419147e+05,6598.018461,"MULTIPOLYGON (((548175.509 177294.512, 548275...."
1602,13892,PROVINCE SUD,Ile des Pins,1.259256,S11666,Valide,2020-08-18,2020-08-18,548458,175581,...,NaN,NaN,NaN,0ha - 10ha ...,2020-08-18,NaT,NaT,1.259256e+04,619.816959,"POLYGON ((548426.971 175686.752, 548456.961 17..."
1751,12265,None,None,3.947643,1032,Valide,2019-01-21,2019-01-21,546611,180302,...,2.0,0.0,9.013878,0ha - 10ha ...,2019-01-21,NaT,NaT,3.947643e+04,1779.468467,"POLYGON ((546593.564 180422.339, 546593.635 18..."
1789,11211,Province Sud,ILE DES PINS,34.698210,1125,Valide,2017-12-27,2017-12-31,549509,179439,...,NaN,NaN,NaN,None,2018-01-06,NaT,NaT,3.469821e+05,6597.926979,"MULTIPOLYGON (((549596.280 179923.944, 549616...."


In [134]:
# Standardisation des champs de dates
data_feux = standardizeField(data_feux_raw, data_date, 'date_id', {'begdate':5, 'enddate':6, 'derniere_detection':21})

In [135]:
data_feux # Visualisation de la table

,objectid,province,commune,surface_ha,idfusion,begdate,enddate,classification,fs_x,fs_y,...,pentemediane,penteminimale,pentemaximale,derniere_detection,tailleincendie,debutviirs,finviirs,shape_starea__,shape_stlength__,geom
117,15865,PROVINCE SUD,Ile des Pins,1.109230,S19698,20201126,20201126,Valide,557910,172898,...,NaN,NaN,NaN,20201126,0ha - 10ha ...,NaT,NaT,1.109230e+04,879.694444,"POLYGON ((557883.772 172965.653, 557933.753 17..."
122,11385,Province Sud,ILE DES PINS,14.311355,941,20170106,20170106,Valide,547895,178828,...,NaN,NaN,NaN,20170106,None,NaT,NaT,1.431136e+05,5638.296097,"POLYGON ((547722.914 179070.783, 547723.057 17..."
923,13509,Province Sud,ILE DES PINS,3.517956,1126,20171117,20171117,Valide,548112,174638,...,NaN,NaN,NaN,20171117,None,NaT,NaT,3.517956e+04,1519.558478,"POLYGON ((548153.733 174755.040, 548153.877 17..."
1038,14190,PROVINCE SUD,Ile des Pins,1.019392,S11647,20200813,20200813,Valide,548490,176350,...,NaN,NaN,NaN,20200813,0ha - 10ha ...,NaT,NaT,1.019392e+04,999.701704,"POLYGON ((548411.811 176406.448, 548441.802 17..."
1257,14512,PROVINCE SUD,Ile des Pins,1.359050,SGroup_90j_82,20200126,20200126,Valide,559470,170398,...,NaN,NaN,NaN,20200126,0ha - 10ha ...,NaT,NaT,1.359050e+04,619.783560,"POLYGON ((559521.892 170388.482, 559521.893 17..."
1381,16237,PROVINCE SUD,Ile des Pins,3.258223,S23691,20201221,20201221,Valide,550116,165331,...,NaN,NaN,NaN,20201221,0ha - 10ha ...,NaT,NaT,3.258223e+04,1679.541881,"POLYGON ((550120.275 165461.609, 550140.269 16..."
1520,10967,Province Sud,ILE DES PINS,14.191474,942,20170106,20170116,Valide,548398,177246,...,NaN,NaN,NaN,20170116,None,NaT,NaT,1.419147e+05,6598.018461,"MULTIPOLYGON (((548175.509 177294.512, 548275...."
1602,13892,PROVINCE SUD,Ile des Pins,1.259256,S11666,20200818,20200818,Valide,548458,175581,...,NaN,NaN,NaN,20200818,0ha - 10ha ...,NaT,NaT,1.259256e+04,619.816959,"POLYGON ((548426.971 175686.752, 548456.961 17..."
1751,12265,None,None,3.947643,1032,20190121,20190121,Valide,546611,180302,...,2.0,0.0,9.013878,20190121,0ha - 10ha ...,NaT,NaT,3.947643e+04,1779.468467,"POLYGON ((546593.564 180422.339, 546593.635 18..."
1789,11211,Province Sud,ILE DES PINS,34.698210,1125,20171227,20171231,Valide,549509,179439,...,NaN,NaN,NaN,20180106,None,NaT,NaT,3.469821e+05,6597.926979,"MULTIPOLYGON (((549596.280 179923.944, 549616...."


### mos_2014

In [45]:
%%time
# Récupétation de la table de MOS de 2014 sur l'emprise souhaitée
data_mos2014 = loadData(catalog,'mos2014').cx[xmin:xmax, ymin:ymax]

CPU times: user 3min 18s, sys: 7.78 s, total: 3min 26s
Wall time: 3min 50s


In [46]:
data_mos2014 # Visualisation de la table

,objectid,idobj,c_2014_n1,c_2014_n2,c_2014_n3,l_2014_n1,l_2014_n2,l_2014_n3,source_14,d_srce_14,...,observ,surface,ombre,d_arbore,d_arbustif,d_herbace,d_autre,shape_length,shape_area,shape
1709,93041,95868.0,3,32,322,Formation végétale,Formation arbustives et/ou herbacées,Strate herbacée,google earth - MOS2010,2013-2014,...,,47663.593545,1,0.000000,0.000000,0.000000,0.000000,1144.953806,47663.593545,"MULTIPOLYGON (((548519.953 172951.173, 548571...."
1728,93096,95869.0,3,32,322,Formation végétale,Formation arbustives et/ou herbacées,Strate herbacée,google earth - MOS2010,2013-2014,...,,44325.845509,1,0.000000,0.000000,0.000000,0.000000,878.398952,44325.845509,"MULTIPOLYGON (((552632.276 176407.105, 552635...."
1759,93108,95870.0,3,32,322,Formation végétale,Formation arbustives et/ou herbacées,Strate herbacée,google earth - MOS2010,2013-2014,...,,13856.919851,1,0.000000,0.000000,0.000000,0.000000,524.770301,13856.919851,"MULTIPOLYGON (((549158.866 177151.494, 549157...."
2376,93709,94796.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,SPOT6,29-06-2013,...,,17773.401176,0,0.976538,79.746354,10.716550,8.560558,813.591432,17773.401176,"MULTIPOLYGON (((551841.674 176281.632, 551833...."
2393,93731,94797.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,SPOT6,29-06-2013,...,,11186.824150,0,1.084991,85.151698,12.899337,0.863974,456.637328,11186.824150,"MULTIPOLYGON (((550660.033 179032.221, 550662...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108655,82807,82592.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,google earth - MOS2010,2013-2014,...,,44539.541432,1,0.000000,0.000000,0.000000,0.000000,992.518110,44539.541432,"MULTIPOLYGON (((550839.980 177518.134, 550842...."
108657,82809,82594.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,google earth - MOS2010,2013-2014,...,,63942.485776,1,0.000000,0.000000,0.000000,0.000000,1150.570784,63942.485776,"MULTIPOLYGON (((546951.984 178905.761, 546966...."
108658,82810,82595.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,google earth - MOS2010,2013-2014,...,,19692.469498,1,0.000000,0.000000,0.000000,0.000000,712.298425,19692.469498,"MULTIPOLYGON (((551244.443 179046.551, 551246...."
108661,82812,82596.0,3,32,321,Formation végétale,Formation arbustives et/ou herbacées,Strate arbustive,google earth - MOS2010,2013-2014,...,,89390.923720,1,0.000000,0.000000,0.000000,0.000000,1681.003653,89390.923720,"MULTIPOLYGON (((551571.290 179008.550, 551569...."


# Indexation des données

## Maillage simple

### communes

In [34]:
%%time
# Indexation des communes sur une grille régulière à l'échelle 8
dim_communes_8 = indexation(data_communes, 8) 

CPU times: user 26.1 s, sys: 0 ns, total: 26.1 s
Wall time: 26.1 s


In [35]:
dim_communes_8 # Visualisation de la table

,objectid,nom,nom_minus,code_com,code_post,nom_fichier,shape_length,shape_area
hex_id,,,,,,,,
889f554f37fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f50b615fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f50b689fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f55436dfffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f55796dfffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
...,...,...,...,...,...,...,...,...
889f50228dfffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08
889f503535fffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08
889f50ad81fffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08


## Maillage adaptatif

### mos_2014

In [96]:
compute_dict_tx(7)

{0: 0.9999999999884721,
 1: 0.9999999999193045,
 2: 0.9999999994351315,
 3: 0.9999999960459205,
 4: 0.9999999723214438,
 5: 0.9999998062501062,
 6: 0.9999986437507449,
 7: 0.9999905062552927,
 8: 0.9999335437870494,
 9: 0.9995348064462535,
 10: 0.9967436451237747,
 11: 0.9772050614067733,
 12: 0.8404428524910452,
 13: 0,
 14: 0,
 15: 0}

In [104]:
%%time
# Indexation du MOS sur une grille adaptative
faits_mos2014_7A12 = compact(gdf=data_mos2014, list_col=['l_2014_n2'], tx_theme=compute_dict_tx(7), tx_spatial=0.95, res_min=7, res_max=10)

/opt/anaconda3/envs/gis_39/lib/python3.9/site-packages/tobler/util/util.py:151: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  source = source.explode()


Résolution 7


/opt/anaconda3/envs/gis_39/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")


InvalidGeometryError: Null geometry supports no operations

In [98]:
faits_mos2014_7A12 # Visualisation de la table

,l_2014_n1
hex_id,
879f54a8bffffff,Formation végétale
879f54a88ffffff,Formation végétale
889f54a157fffff,Formation végétale
889f54a139fffff,Formation végétale
889f54a13dfffff,Formation végétale
...,...
8a9f54aa32c7fff,Formation végétale
8a9f54aa3237fff,Formation végétale
8a9f54a8588ffff,Formation végétale


# Intégration des données

## Tables de dimensions

### communes

In [36]:
%%time
# Intégration de la table dans le DWH
updateTable(dim_communes_8, 'dim_communes_8', engine, 'bilbo', methode='replace', geom=False)

CPU times: user 998 ms, sys: 0 ns, total: 998 ms
Wall time: 2.05 s


,objectid,nom,nom_minus,code_com,code_post,nom_fichier,shape_length,shape_area
hex_id,,,,,,,,
889f554f37fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f50b615fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f50b689fffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f55436dfffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
889f55796dfffff,1,MONT DORE,Mont Dore,98817,98809,MONT_DORE,400107.916505,6.361819e+08
...,...,...,...,...,...,...,...,...
889f50228dfffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08
889f503535fffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08
889f50ad81fffff,33,THIO,Thio,98829,98829,THIO,303539.628921,9.889216e+08


In [42]:
# Génération d'une vue ajoutant une colonne de géométrie à la table
geomView(table_name='dim_communes_8', engine=engine, schema='bilbo')

## Tables de faits

### incendies_Sentinel

In [130]:
%%time
# Intégration de la table dans le DWH
updateTable(faits_feux_13, 'faits_feux_13', engine, 'bilbo', methode='replace', geom=False)

CPU times: user 6.56 s, sys: 23.4 ms, total: 6.59 s
Wall time: 12 s


,objectid,province,commune,surface_ha,idfusion,classification,begdate,enddate,fs_x,fs_y,...,pentemoyenne,pentemediane,penteminimale,pentemaximale,tailleincendie,derniere_detection,debutviirs,finviirs,shape_starea__,shape_stlength__
hex_id,,,,,,,,,,,,,,,,,,,,,
8d9f54aac8095bf,15865,PROVINCE SUD,Ile des Pins,1.10923,S19698,Valide,20201126,20201126,557910,172898,...,NaN,NaN,NaN,NaN,0ha - 10ha ...,20201126,NaT,NaT,11092.295708,879.694444
8d9f54aac80e8bf,15865,PROVINCE SUD,Ile des Pins,1.10923,S19698,Valide,20201126,20201126,557910,172898,...,NaN,NaN,NaN,NaN,0ha - 10ha ...,20201126,NaT,NaT,11092.295708,879.694444
8d9f54aac80e8ff,15865,PROVINCE SUD,Ile des Pins,1.10923,S19698,Valide,20201126,20201126,557910,172898,...,NaN,NaN,NaN,NaN,0ha - 10ha ...,20201126,NaT,NaT,11092.295708,879.694444
8d9f54aac80e6ff,15865,PROVINCE SUD,Ile des Pins,1.10923,S19698,Valide,20201126,20201126,557910,172898,...,NaN,NaN,NaN,NaN,0ha - 10ha ...,20201126,NaT,NaT,11092.295708,879.694444
8d9f54aac80c77f,15865,PROVINCE SUD,Ile des Pins,1.10923,S19698,Valide,20201126,20201126,557910,172898,...,NaN,NaN,NaN,NaN,0ha - 10ha ...,20201126,NaT,NaT,11092.295708,879.694444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8d9f54af57a363f,11163,Province Sud,ILE DES PINS,1.79896,1157,Valide,20171227,20171227,548288,173846,...,NaN,NaN,NaN,NaN,None,20171227,NaT,NaT,17989.599846,1039.699343
8d9f54af578d07f,11163,Province Sud,ILE DES PINS,1.79896,1157,Valide,20171227,20171227,548288,173846,...,NaN,NaN,NaN,NaN,None,20171227,NaT,NaT,17989.599846,1039.699343
8d9f54af57aa2ff,11163,Province Sud,ILE DES PINS,1.79896,1157,Valide,20171227,20171227,548288,173846,...,NaN,NaN,NaN,NaN,None,20171227,NaT,NaT,17989.599846,1039.699343


### mos_2014

In [99]:
%%time
# Intégration de la table dans le DWH
updateTable(faits_mos2014_7A12, 'faits_mos2014_test', engine, 'bilbo', methode='replace', geom=False)

CPU times: user 63.9 ms, sys: 2.96 ms, total: 66.9 ms
Wall time: 990 ms


,l_2014_n1
hex_id,
879f54a8bffffff,Formation végétale
879f54a88ffffff,Formation végétale
889f54a157fffff,Formation végétale
889f54a139fffff,Formation végétale
889f54a13dfffff,Formation végétale
...,...
8a9f54aa32c7fff,Formation végétale
8a9f54aa3237fff,Formation végétale
8a9f54a8588ffff,Formation végétale


In [100]:
# Génération d'une vue ajoutant une colonne de géométrie à la table
geomView(table_name='faits_mos2014_test', engine=engine, schema='bilbo')